## API Gallica SRU :  <i>Les trois mousquetaires</i>

Le script suit le scénario de recherche documentaire décrit dans ce [tutoriel vidéo](https://www.youtube.com/watch?v=SF0Yx0Tg8vA) Gallica.

### 1. Construire la requête et appeler l'API SRU Gallica

In [98]:
# URL de base du service Gallica SRU
BASEURL = 'https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query='
# L'API SRU renvoie au maximum 50 documents par appel
MODULE = 50
# Mode d'agrégation des résultats : ne pas agréger les documents multivolumes
COLLAPSING = "false"

## Exemple de requête pour "Les trois mousquetaires" ##
# 1. Construire la requête Gallica en s'aidant du formulaire de recherche avancé de Gallica
#  titre = Les trois mousquetaires
#  auteur = Dumas, Alexandre
#  type = Livres
#  liste = Au volume/fascicule
# 2. Lancer la requête
# 3. Dans l'URL Gallica, recopier la requête CQL qui suit le paramètre query=  
query = '%28%28dc.creator%20all%20%22Dumas%2C%20Alexandre%22%20or%20dc.contributor%20all%20%22Dumas%2C%20Alexandre%22%20%29%20and%20dc.title%20all%20%22les%20trois%20mousquetaires%22%20%29%20and%20%28dc.type%20all%20%22monographie%22%29'

# construire la reuqête avec ses paramètres
req_url = "".join([BASEURL, query, "&maximumRecords=", str(MODULE), "&collapsing=", COLLAPSING])


import requests

# Appel de l'API SRU avec le système de pagination : i = [1-n]
def getXML_from_SRU(indice):
    req = "".join([req_url,"&startRecord=",str(indice)])
    print ("... appel de l'API Gallica SRU : ", req)
    temp = requests.get(req)
    #print (temp.status_code)
    if temp.status_code != 200:
        print ("... oups, erreur API:", temp.status_code)
        return
    else:
        return temp

print ("... extraction de la page de résultats #", 1)
r = getXML_from_SRU(1)        
if r is not None:
    print (r.text[0:1000])

... extraction de la page de résultats # 1
... appel de l'API Gallica SRU :  https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=%28%28dc.creator%20all%20%22Dumas%2C%20Alexandre%22%20or%20dc.contributor%20all%20%22Dumas%2C%20Alexandre%22%20%29%20and%20dc.title%20all%20%22les%20trois%20mousquetaires%22%20%29%20and%20%28dc.type%20all%20%22monographie%22%29&maximumRecords=50&collapsing=true&startRecord=1
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<srw:searchRetrieveResponse xmlns:ns5="http://gallica.bnf.fr/namespaces/gallica/" xmlns:oai_dc="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:srw="http://www.loc.gov/zing/srw/" xmlns:dc="http://purl.org/dc/elements/1.1/">
    <srw:version>1.2</srw:version>
    <srw:echoedSearchRetrieveRequest>
        <srw:query>((dc.creator all "Dumas, Alexandre" or dc.contributor all "Dumas, Alexandre" ) and dc.title all "les trois mousquetaires" ) and (dc.type all "monographie")</srw:query>
        <srw:version>1.2</srw:vers

### 2. Parser le flux XML pour obtenir les données

#### a. Il faut d'abord obtenir le nombre de documents 

In [99]:
from bs4 import BeautifulSoup as bs
bs_content = bs(r.text, "xml") # bizarre : avec le parser xml, on perd la fin du contenu XML ...
#bs_content = bs(r.text, "lxml") # par contre lxml ne fonctionne pas sur srw:numberOfRecords !

# Extraire le nombre de résultats
n_docs = int(bs_content.find("srw:numberOfRecords").getText())
print("Nombre de résultats : ", n_docs)

Nombre de résultats :  11


#### b. Il faut maintenant paginer le flot XML restant

In [100]:
bs_content = bs(r.text, "lxml")

# si besoin, on pagine la suite
if n_docs > MODULE:
    print (n_docs)
    # Ajouter au document xml toutes les réponses des autres pages
    for i in range(1, int(n_docs/MODULE)+1):
        print ("Extraction de la page de résultats #", i+1) 
        print ("    startRecord : ", i*MODULE+1)
        temp = getXML_from_SRU(i*MODULE+1)
        temp_xml = bs(temp.text, "lxml")
        bs_content.append(temp_xml)

print ("------------")        
print("Nbre total d'identifiants de document : ",len(bs_content.find_all("dc:identifier")))
for ark in bs_content.find_all("uri"):
    print(ark.get_text())

------------
Nbre total d'identifiants de document :  11
bpt6k55511660
bpt6k9661806g
bpt6k61336787
bpt6k1073528t
bpt6k55894240
bpt6k61407607
bpt6k202872k
bpt6k1166645k
bpt6k690847
bpt6k61386425
bpt6k1521259j


#### C. Exploitation des données XML

In [101]:
for ark in bs_content.find_all("dc:title"):
    print(ark.get_text())

Les trois mousquetaires / [par Alexandre Dumas]
Les trois mousquetaires. Tome 1 / Alexandre Dumas ; avec une lettre d'Alexandre Dumas fils ; compositions de Maurice Leloir, gravures sur bois de J. Huyot,...
Les Trois mousquetaires, par M. Alexandre Dumas
Les Trois mousquetaires. Tome 7-8 / par Alexandre Dumas
Les trois mousquetaires ; Vingt ans après ; Vicomte de Bragelonne. 1 / Alexandre Dumas
Vingt ans après, suite des "Trois mousquetaires. Tome 3 / par Alexandre Dumas
Oeuvres illustrées ; 1. Les trois mousquetaires. 1, 2 / par Alexandre Dumas ; ill. par J. A. Beaucé, F. Philippoteaux, etc.
Trois mousquetaires.... Partie 2 / Dessins de Jean Simard
Vingt ans après : suite des "Trois mousquetaires". 1 / par Alexandre Dumas
Vingt ans après, suite des "Trois mousquetaires", par M. Alexandre Dumas
Le Vicomte de Bragelonne, ou Dix ans plus tard. Tome 21 / , suite des "Trois mousquetaires" et de "Vingt ans après". [Histoire contemporaine.] Par Alexandre Dumas
